# Instructions
The following code was designed in order to load in and score motion/freezing data from video files. It allows for cropping of the video frame to reduce influence of miniscope/optogenetic cables.  After initally loading in the video, the user is able to crop the video frame by adjusting a single variable.  Motion is then detected by capturing the number of pixels whose frame by frame grayscale change value exceeds a threshold (determined using FreezeAnalysis_Calibration.ipynb).  Freezing is then assessed by calculating when motion drops below a user-defined threshold.  The user is able to visualize raw video, animal motion, and whether an animal is deemed to be freezing in order optimize parameters.  Final output is a csv which provides frame by frame motion and freezing across the session.  Additionally, summary file can be generated which allows user to get motion/freezing for specified time bins.  With the exception of setting the file path and small number of parameters (ie frames per second in video, freezing threshold), user can run through code.  Once parameters are found that work well, batch processing of multiple videos can be performed using FreezeAnalysis_BatchProcess.ipynb.

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

---
# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import holoviews as hv
import FreezeAnalysis_Functions as fz

---
# 2. User Sets Directory and File Information
Below the user sets the directory file path (path of the folder where the video file is), the file name of the video to be analyzed, the frame rate of the video, the frame on which the analysis is to begin (0 is the first frame), and the last frame to be analyzed (set `'end' : None` if processing entire video).  Additionally, the user defines the names of any regions of interest.  Any number of regions of interst can be used.  If no regions of interest are to be used, set `region_names = None`.
***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [ ]:
video_dict = {
    'dpath' : "/Users/zp/Videos", # directory containing file
    'file' : "video.mp4", #filename with extension
    'fps' : 30, #frames per second
    'start' : 0, #frame at which to start. 0-based
    'end' : None #frame at which to end.  set to None if processing whole video.
}

---
# 3. Load Video and Crop Frame if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

In [ ]:
%%output size=100
#Select output size if image is too small/large.  Code above must be first line in cell and dictates overall size
#of image, where 100 is standard.  stretch_w and stretch_h below allow you to alter relative width to height ratio.

#Get full path and first frame
image,crop,video_dict=fz.LoadAndCrop(video_dict,cropmethod="Box")
image

---
# 4. Analyze Motion Across Session

### 4a. User Sets Parameters for Motion Analysis

In [ ]:
mt_cutoff = 10 #grayscale difference value required for pixel to be counted as changing

### 4b. Detect Motion and Plot
Here, code loops through all frames and detects number of pixels whose grayscale change exceeds cutoff per frame.  Motion vector is populated with these values.  In addition to interactive plot options via toolbar, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [ ]:
Motion = fz.Measure_Motion(video_dict,mt_cutoff,crop=crop,SIGMA=1)  #calls function to get motion

#Plot motion
h,w = 300,1000 #height,width
motion_plot = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
    height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
motion_plot

---
# 5. Analyze Session Freezing

### 5a. User Selects Freezing Parameters

In [ ]:
FreezeThresh = 500 #Upper bound for freezing, in frame-by-frame pixels changed
MinDuration = 15 #Number of frames motion must be below threshold to begin accruing freezing

### 5b. Measure Freezing and Save
Motion vector is populated with these values.  In addition to interactive plot options via toolbar, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [ ]:
%%output size=100

#Calculate Freezing
Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
print('Average Freezing: {x}%'.format(x=np.average(Freezing)))
fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)

#Plot Freezing
h,w = 300,1000 #height,width
freezing_plot = hv.Area(Freezing*(Motion.max()/100),'Frame','Motion').opts(
    color='lightgray',line_width=0,line_alpha=0)
motion_plot = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Motion').opts(
    height=h,width=w,line_width=1, color='steelblue',title="Motion Across Session with Freezing Highlighted in Gray")
freezing_plot*motion_plot

---
# 6. (Optional) Display Section of Video with Motion and Freezing
After analyzing freezing a section of the video can be replayed and the animal's state - "Active/Freezing" - will be displayed.  

In [ ]:
#Video parameters
display_dict = {
    'start' : 0, #start point of video segment in frames.  0 if beginning of video.
    'end' : 100, #end point of video segment in frames. this is NOT the duration of the segment
    'save_video' : False #Option to save video if desired.  Currently will be saved at 20 fps even if video is something else
}

fz.PlayVideo(video_dict,display_dict,Freezing,mt_cutoff,crop=crop,SIGMA=1)


---
# 7. (Optional) Create Binned Summary Report and Save
The code below allows the user to either save a csv containing summary data for user-defined bins (e.g. % time freezing for each minute) or a session-wide average. 

***If you only want a session avg***, set `bin_dict = None`

***If you are not using ROIs***, in the code below, set value of 'region_names' within function to None: `region_names=None`.  Otherwise, keep `region_names=region_names`

To specify bins, set bin_dict using the following notation, where start and stop represent time in seconds:

```
bin_dict = {
    'BinName1': (start, stop),
    'BinName2': (start, stop),
    'BinName3': (start, stop),
}
```

In [ ]:
bin_dict = {
    1: (0,60),
    2: (60,120),
    3: (120,180)
}

summary = fz.Summarize(video_dict,Motion,Freezing,FreezeThresh,MinDuration,mt_cutoff,bin_dict=bin_dict)
summary.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_SummaryStats.csv')
summary